## Joshua Muzei (jmuzei2)
## IE598 MLF F19
## Module 6 Homework (Cross validation)


In [1]:
import pandas as pd
import ipywidgets
from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import timeit
%matplotlib inline

In [2]:
df = pd.read_excel('default of credit card clients.xls',  header = 1)
pd.set_option('display.max_columns', None)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [3]:
df = df.iloc[:,1:]

In [4]:
df['EDUCATION'].unique()

array([2, 1, 3, 5, 4, 6, 0], dtype=int64)

In [5]:
df['MARRIAGE'].unique()

array([1, 2, 3, 0], dtype=int64)

In [6]:
df['SEX'].unique()

array([2, 1], dtype=int64)

In [7]:
#col_names = ['EDUCATION','MARRIAGE','SEX','default payment next month']
#for col in col_names:
    #df[col] = df[col].astype('category',copy=False)

In [8]:
df.shape

(30000, 24)

In [9]:
# Data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   LIMIT_BAL                   30000 non-null  int64
 1   SEX                         30000 non-null  int64
 2   EDUCATION                   30000 non-null  int64
 3   MARRIAGE                    30000 non-null  int64
 4   AGE                         30000 non-null  int64
 5   PAY_0                       30000 non-null  int64
 6   PAY_2                       30000 non-null  int64
 7   PAY_3                       30000 non-null  int64
 8   PAY_4                       30000 non-null  int64
 9   PAY_5                       30000 non-null  int64
 10  PAY_6                       30000 non-null  int64
 11  BILL_AMT1                   30000 non-null  int64
 12  BILL_AMT2                   30000 non-null  int64
 13  BILL_AMT3                   30000 non-null  int64
 14  BILL_A

In [10]:
print(df.isnull().sum())

LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default payment next month    0
dtype: int64


In [11]:
# Data summary
df.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,-0.266200,-0.291100,51223.330900,49179.075167,4.701315e+04,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,1.133187,1.149988,73635.860576,71173.768783,6.934939e+04,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3558.750000,2984.750000,2.666250e+03,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22381.500000,21200.000000,2.008850e+04,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,67091.000000,64006.250000,6.016475e+04,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


In [12]:
# Box plot : select the variable name to see the corresponding plots
import warnings
warnings.filterwarnings("ignore")

    
var_name = df.columns.to_list()
@ipywidgets.interact(xyname = var_name)
def box_plot(xyname):
    sns.boxplot(df[xyname])


interactive(children=(Dropdown(description='xyname', options=('LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AG…

## Using the ccdefault dataset, with 90% for training and 10% for test (stratified sampling) and the decision tree model that you did in Module 2:


In [13]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1].values
y = df['default payment next month'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y)

In [14]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Part 1: Random test train splits
Run in-sample and out-of-sample accuracy scores for 10 different samples by changing random_state from 1 to 10 in sequence. 
Display the individual scores, then calculate the mean and standard deviation on the set of scores.  Report in a table format.


In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Loop through 10 different decision tree classifiers with different random states
random_state_list = np.arange(1,11)
for i in random_state_list:    
    tree = DecisionTreeClassifier(criterion='gini', 
                                  max_depth=4, 
                                  random_state=i)
    tree.fit(X_train_std, y_train)

    # Make predictions on the training data
    y_pred = tree.predict(X_train_std)
    
    # Calculate the accuracy of the tree classifier with train data
    accuracy = accuracy_score(y_train, y_pred)
    print(f"In Sample Accuracy with random state {i}: {accuracy}")
    
    # Make predictions on the test data
    y_pred = tree.predict(X_test_std)
    
    # Calculate the accuracy of the tree classifier with train data
    accuracy = accuracy_score(y_test, y_pred)
    print(f" Out of Sammple Accuracy with random state {i}: {accuracy}")
    

In Sample Accuracy with random state 1: 0.8234444444444444
 Out of Sammple Accuracy with random state 1: 0.822
In Sample Accuracy with random state 2: 0.8234444444444444
 Out of Sammple Accuracy with random state 2: 0.822
In Sample Accuracy with random state 3: 0.8234444444444444
 Out of Sammple Accuracy with random state 3: 0.822
In Sample Accuracy with random state 4: 0.8234444444444444
 Out of Sammple Accuracy with random state 4: 0.822
In Sample Accuracy with random state 5: 0.8234444444444444
 Out of Sammple Accuracy with random state 5: 0.822
In Sample Accuracy with random state 6: 0.8234444444444444
 Out of Sammple Accuracy with random state 6: 0.822
In Sample Accuracy with random state 7: 0.8234444444444444
 Out of Sammple Accuracy with random state 7: 0.822
In Sample Accuracy with random state 8: 0.8234444444444444
 Out of Sammple Accuracy with random state 8: 0.822
In Sample Accuracy with random state 9: 0.8234444444444444
 Out of Sammple Accuracy with random state 9: 0.822
I

## Part 2: Cross validation
Now rerun your model using cross_val_scores with k-fold CV (k=10).  
Report the individual fold accuracy scores, the mean CV score and the standard deviation of the fold scores.  Now run the out-of-sample accuracy score.  Report in a table format.


In [16]:
from sklearn.model_selection import StratifiedKFold
    

kfold = StratifiedKFold(n_splits=10,
                        random_state=None).split(X_train_std, y_train)

scores = []
for k, (train, test) in enumerate(kfold):
    tree.fit(X_train_std[train], y_train[train])
    score = tree.score(X_train_std[test], y_train[test])
    scores.append(score)
    print(f"Fold {k+1} accuracy score: {score}")
    

# Calculate the mean and standard deviation of the fold scores
mean_score = np.mean(scores)
std_score = np.std(scores)

# Print out the mean and standard deviation of the fold scores
print(f"Mean cross-validation score: {mean_score}")
print(f"Standard deviation of fold scores: {std_score}")


Fold 1 accuracy score: 0.8181481481481482
Fold 2 accuracy score: 0.8181481481481482
Fold 3 accuracy score: 0.812962962962963
Fold 4 accuracy score: 0.8107407407407408
Fold 5 accuracy score: 0.8233333333333334
Fold 6 accuracy score: 0.8277777777777777
Fold 7 accuracy score: 0.8196296296296296
Fold 8 accuracy score: 0.8251851851851851
Fold 9 accuracy score: 0.8159259259259259
Fold 10 accuracy score: 0.8251851851851851
Mean cross-validation score: 0.8197037037037036
Standard deviation of fold scores: 0.005323035325398988


## Part 3: Conclusions

We preprocessed and scale-transformed the dataset to make sure all the attribubes have almost the same scale. We devided the dataset into training (90%) and testing (10%). in our first approach, we fitted a Decision tree classifier to the training data and evaluated the performance of our model on the testing data (Holdout approach). We also changed the random-state from 1 to 10 to assess how the in and out of sample accuracy will change. The results are the same for the in-sample accuracy (0.823) and the out-sample accuracy (0.822). In other words, the change in the random state did not affect the model performance. 
Ww used 10 folds cross-validation in our second approach. The accuracy of the model changed for all the 10 folds. The average accuracy score is 0.82 and the standard deviation of the 10 folds is 0.005. 
The hodout method provide the best accuracy (0.822) compared to the cross-validation method (0.819+/-0.05). In fact, they are almost the same. The cross-validation method is robust. However, it is computationally very demanding.

In [17]:
print("My name is {Muzei Joshua}")
print("My NetID is: {jmuzei2}")
print("I hereby certify that I have read the University policy on Academic Integrity and that I am not in violation.")


My name is {Muzei Joshua}
My NetID is: {jmuzei2}
I hereby certify that I have read the University policy on Academic Integrity and that I am not in violation.
